In [1]:
from tensorflow import keras
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.0,
                                                       vertical_flip = True,
                                                       horizontal_flip = True)

In [2]:
from keras.applications import ResNet50

conv_base = ResNet50(weights = "imagenet",
include_top = False,
input_shape = (224, 224, 3))
conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [3]:
trainGen = datagen.flow_from_directory(
    r'C:\Users\vidha\Health Analytics\archive\Covid19-dataset\train',
    target_size = (224,224),
    batch_size = 32,
    shuffle = True,
    class_mode = 'categorical'
)

datatestgen = keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.0)
                                                       
testGen = datatestgen.flow_from_directory(
    r'C:\Users\vidha\Health Analytics\archive\Covid19-dataset\test',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [4]:
import numpy as np
def extractfeatures(sample_count, generator):
    i = 0
    batch_size = 32
    features = np.zeros(shape=(sample_count, 7, 7, 2048))
    labels = np.zeros(shape=(sample_count, 3))
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i+1) * batch_size] = features_batch
        labels[i * batch_size: (i+1) * batch_size] = labels_batch
        i += 1
        if ((i * batch_size % 1000) == 0 ):
            print("processed size =", i * batch_size)
        if i * batch_size >= sample_count:
            break
    return features, labels

In [5]:
train_features, train_labels = extractfeatures(251, trainGen)
test_features, test_labels = extractfeatures(66, testGen)

1/1 [==============================] - 1s 690ms/step


In [11]:
classifier = keras.Sequential([
    keras.layers.Flatten(input_shape=(7, 7, 2048)),
    keras.layers.Dense(256, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation = 'softmax'),
])
classifier.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

In [12]:
classifier.fit(train_features, train_labels, steps_per_epoch = 251 // 32, epochs = 20, validation_split = 0.2)

Epoch 1/20
7/7 [==============================] - 10s 853ms/step - loss: 19.7669 - accuracy: 0.3700 - val_loss: 0.6302 - val_accuracy: 0.6667
Epoch 2/20
7/7 [==============================] - 5s 711ms/step - loss: 2.4791 - accuracy: 0.3650 - val_loss: 0.6356 - val_accuracy: 0.8039
Epoch 3/20
7/7 [==============================] - 5s 736ms/step - loss: 0.9391 - accuracy: 0.5600 - val_loss: 0.7956 - val_accuracy: 0.5490
Epoch 4/20
7/7 [==============================] - 4s 632ms/step - loss: 0.9165 - accuracy: 0.5550 - val_loss: 0.7618 - val_accuracy: 0.5882
Epoch 5/20
7/7 [==============================] - 4s 646ms/step - loss: 0.9046 - accuracy: 0.5050 - val_loss: 0.7276 - val_accuracy: 0.6667
Epoch 6/20
7/7 [==============================] - 4s 638ms/step - loss: 0.9052 - accuracy: 0.4950 - val_loss: 0.7628 - val_accuracy: 0.6471
Epoch 7/20
7/7 [==============================] - 4s 597ms/step - loss: 0.8448 - accuracy: 0.5350 - val_loss: 0.7237 - val_accuracy: 0.6471
Epoch 8/20
7/7 [==

In [13]:
classifier.save('ResNet50FinalProject.model')

INFO:tensorflow:Assets written to: ResNet50FinalProject.model\assets


INFO:tensorflow:Assets written to: ResNet50FinalProject.model\assets


In [14]:
classifier.evaluate(test_features, test_labels, batch_size = 32)

3/3 [==============================] - 0s 58ms/step - loss: 0.6680 - accuracy: 0.6970


[0.6680081486701965, 0.6969696879386902]

In [15]:
classifier.evaluate(train_features, train_labels)

8/8 [==============================] - 1s 106ms/step - loss: 0.6733 - accuracy: 0.7052


[0.673261821269989, 0.7051792740821838]